In [1]:
w = {}
import os
import tensorflow as tf
from dqn.ops import linear
from dqn.utils import get_time, save_pkl, load_pkl
from dqn.ops import conv2d
import numpy as np
obs=np.array([[[ 1.],
        [ 0.]],

       [[ 0.],
        [ 1.]],

       [[ 1.],
        [ 0.]],

       [[ 0.],
        [ 1.]],

       [[ 0.],
        [ 1.]],

       [[ 0.],
        [ 1.]],

       [[ 0.],
        [ 1.]],

       [[ 0.],
        [ 1.]]])

In [2]:
w1 = np.expand_dims(np.array([[[1.,0.],[0., 1.]],[[1.,0.],[0.,1.]]]), axis=2)
print w1
obs = obs.transpose((2,0,1))

[[[[ 1.  0.]]

  [[ 0.  1.]]]


 [[[ 1.  0.]]

  [[ 0.  1.]]]]


In [4]:
with tf.Session() as sess:
    w = {}
    t_w = {}

    init1_w = tf.constant_initializer(w1)
    init1_b = tf.constant_initializer(-1.0)
    init2_w = tf.constant_initializer(1.) 
    init2_b = tf.constant_initializer(0.0)
    activation_fn = tf.nn.relu

    with tf.variable_scope('prediction', reuse=None):
        s_t = tf.placeholder('float32',
                    [None, 1, 8, 2], name='s_t')
        l1, w['l1_w'], w['l1_b'] = conv2d(s_t, 
                    2, [2, 2], [2, 1], init1_b, init1_w, activation_fn, 'NCHW', name='l1' )
        l2, w['l2_w'], w['l2_b'] = conv2d(l1, 
                    1, [1, 1], [1, 1], init2_b, init2_w, activation_fn, 'NCHW', name='l2' )
        
        shape = l2.get_shape().as_list()
        l2_flat = tf.reshape(l2, [-1, reduce(lambda x, y: x * y, shape[1:])])
        q, w['q_w'], w['q_b'] = linear(l2_flat, 5, activation_fn=activation_fn, name='q')
        q_action = tf.argmax(q, dimension=1)

    tf.initialize_all_variables().run()
    action = q_action.eval({s_t: [obs]})[0]
    l1 = l2.eval({s_t: [obs]})[0]
    q_w = w['q_w'].eval({s_t: [obs]})[0]
    print l1
    print q_w
    print action

AttributeError: 'dict' object has no attribute 'graph'

In [4]:
w

{'l1_b': <tensorflow.python.ops.variables.Variable at 0x7fc361f325d0>,
 'l1_w': <tensorflow.python.ops.variables.Variable at 0x7fc361f32110>,
 'l2_b': <tensorflow.python.ops.variables.Variable at 0x7fc3082e1dd0>,
 'l2_w': <tensorflow.python.ops.variables.Variable at 0x7fc361f32a90>}

In [5]:
with tf.Session() as sess:
    with tf.variable_scope('load_pred_from_pkl', reuse=True):
        w_input = {}
        w_assign_op = {}

        for name in w.keys():
            w_input[name] = tf.placeholder('float32', w[name].get_shape().as_list(), name=name)
            w_assign_op[name] = w[name].assign(w_input[name])

        for name in w.keys():
            print name
            print w_assign_op[name].eval({w_input[name]: load_pkl(os.path.join('weights', "%s.pkl" % name))})



l2_b
  [*] load weights/l2_b.pkl
   [-] load_pkl : 0.00020 sec
[ 1.30245376]
l1_w
  [*] load weights/l1_w.pkl
   [-] load_pkl : 0.00009 sec
[[[[ 2.28910303  0.28293025]]

  [[ 0.28559086  2.28874159]]]


 [[[ 2.28989935  0.28293025]]

  [[ 0.28559086  2.28695154]]]]
l2_w
  [*] load weights/l2_w.pkl
   [-] load_pkl : 0.00008 sec
[[[[ 2.3441813 ]
   [ 2.33947611]]]]
l1_b
  [*] load weights/l1_b.pkl
   [-] load_pkl : 0.00006 sec
[ 0.29399195  0.29088825]
